In [1]:
!pip install pyspark
!pip install findspark

In [41]:
import json
import requests
from pyspark.sql import SparkSession

In [42]:
# Function to read data from the API
def read_api(url):
    response = requests.get(url)
    return response.text

In [43]:
url = "jdbc:mysql://127.0.0.1:3306/sau"
mysql_properties = {
    "user": "root",
    "password": "Shivramsriramulu1234",
    "driver": "com.mysql.jdbc.Driver"
}


In [44]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySpark MySQL Connection") \
    .config("spark.jars", "/Users/shivramsriramulu/Downloads/mysql-connector-java-8.0.27.jar") \
    .getOrCreate()

In [4]:
spark

In [45]:
# API URL
api_url = r"https://api.coindesk.com/v1/bpi/currentprice.json"

In [46]:
api_data = read_api(api_url)
payload = json.loads(api_data)

# Extract relevant data
bpi_data = payload.get("bpi", {})

In [47]:
# Create PySpark DataFrame
data = [(key, value.get("rate_float")) for key, value in bpi_data.items()]
columns = ["currency", "rate"]
df = spark.createDataFrame(data, columns)

In [50]:
df.show()

+--------+----------+
|currency|      rate|
+--------+----------+
|     USD|42678.6367|
|     GBP|35661.9274|
|     EUR|41575.2232|
+--------+----------+



In [12]:
import requests, json
def read_api(url: str):
    normalized_data = dict()
    data = requests.get(api_url).json() 
    normalized_data["_data"] = data # Normalize payload to handle array situtations
    return json.dumps(normalized_data)

In [49]:
# Write DataFrame to MySQL
df.write.jdbc(url=url, table="tripdata", mode="overwrite", properties=mysql_properties)